# Best Neighborhood to Rent in Hamilton 

## Introduction

Hamilton is a Canadian port city on the western tip of Lake Ontario, it is famous for its amazing escarpment views and numerous waterfalls, it is a great place to live in. Many people working in Toronto are moving to Hamilton because of the great natural views, cheaper rent and convenient transportation. The objective of this project is to analyze the neighborhoods in Hamilton based on the featured venues and the average rent price to help people who are considering moving to Hamilton to find the best place to rent that suit their needs. 

## Table of Contents


1. Download and Explore Dataset 

2. Explore Neighborhoods in Hamilton, ON 

3. Analyze Each Neighborhood 

4. Cluster Neighborhoods

5. Examine Clusters 

First, download the libraries needed for the project.

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Download and Explore Dataset

### 1.1 Get the Rent Dataframe

In [4]:
rent_data = {
'Durand' : 1300,
'Central Hamilton':1675,
'Landsdale': 1345,
'Beasley': 1395,
'Corktown': 1375,
'Gibson': 1199,
'Kirkendall North': 1720,
'Stipley': 1175,
'Ainslie Wood East': 525,
'Westdale South': 720,
'Riverdale West': 1450,
'Crown Point West': 1195,
'Stinson': 1310,
'Strathcona': 1272,
'St. Clair': 1250,
'Rosedale': 1424,
'Raleigh': 1348,
'Crown Point East': 1100,
'North End East': 1200,
'Waterdown': 2150,
'Ainslie Wood West': 1129,
'Ainslie Wood North': 560,
'University Gardens': 1375,
'Hill Park': 1115,
'Stoney Creek Estates': 949,
'Bakeley': 1320,
'Greenford': 1650}

In [87]:
rent_df = pd.DataFrame.from_dict(rent_data,orient='index').reset_index()
rent_df.columns = ['Neighbourhood','MedianRent']
rent_df

,Neighbourhood,MedianRent
0,Durand,1300
1,Central Hamilton,1675
2,Landsdale,1345
3,Beasley,1395
4,Corktown,1375
5,Gibson,1199
6,Kirkendall North,1720
7,Stipley,1175
8,Ainslie Wood East,525
9,Westdale South,720


### 1.2 Get the GIS location data

In [88]:
geolocator = Nominatim(user_agent = 'My app')
def getLat(address):
    loc = address + ', Hamilton, Canada'
    if geolocator.geocode(loc):
        return geolocator.geocode(loc).latitude 
def getLon(address):
    loc = address + ', Hamilton, Canada'
    if geolocator.geocode(loc):
        return geolocator.geocode(loc).longitude
rent_df['Latitude'] = rent_df['Neighbourhood'].apply(getLat) 
rent_df['Longitude'] = rent_df['Neighbourhood'].apply(getLon) 

In [89]:
rent_df.dropna(subset = ['Latitude'],inplace = True)

In [91]:
rent_df.reset_index(drop=True, inplace = True)
rent_df

,Neighbourhood,MedianRent,Latitude,Longitude
0,Durand,1300,43.250247,-79.875734
1,Central Hamilton,1675,43.256080,-79.872858
2,Beasley,1395,43.259204,-79.861012
3,Corktown,1375,43.250681,-79.868619
4,Gibson,1199,43.257866,-79.839098
5,Kirkendall North,1720,39.977308,-86.047118
6,Westdale South,720,43.261881,-79.905921
7,Riverdale West,1450,43.228332,-79.759820
8,Stinson,1310,43.246953,-79.852747
9,Strathcona,1272,43.265244,-79.883693


#### Create a map of Hamilton with neighborhoods superimposed on top.

In [10]:
address = 'Hamilton, ON, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.2560802, -79.8728583.


In [11]:
# create map of New York using latitude and longitude values
map_hamilton = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, neighborhood in zip(rent_df['Latitude'], rent_df['Longitude'], rent_df['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hamilton)  
    
map_hamilton

Next, start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = '2CFPS0RAJBRO5QUJ0TAVVPUDLK0LHDDBYQHKZ4RRSDN5XEPH' # your Foursquare ID
CLIENT_SECRET = '2CGAOWN5Z5CQPZNSFUNJEK5QUR33WMXX1KB1OZBDAQPOVBUM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 2CFPS0RAJBRO5QUJ0TAVVPUDLK0LHDDBYQHKZ4RRSDN5XEPH
CLIENT_SECRET:2CGAOWN5Z5CQPZNSFUNJEK5QUR33WMXX1KB1OZBDAQPOVBUM


## 2. Explore Neighborhoods in Hamilton, ON

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each neighborhood and create a new dataframe called *hamilton_venues*.

In [14]:
LIMIT = 100
hamilton_venues = getNearbyVenues(names=rent_df['Neighbourhood'],
                                   latitudes=rent_df['Latitude'],
                                   longitudes=rent_df['Longitude']
                                  )


Durand
Central Hamilton
Beasley
Corktown
Gibson
Kirkendall North
Westdale South
Riverdale West
Stinson
Strathcona
St. Clair
Rosedale
Raleigh
North End East
Waterdown
University Gardens
Hill Park
Greenford


In [113]:
hamilton_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Durand,43.250247,-79.875734,One Duke Restaurant and Lounge,43.251812,-79.871852,Seafood Restaurant
1,Durand,43.250247,-79.875734,Red Crow Coffee,43.250061,-79.871915,Café
2,Durand,43.250247,-79.875734,Durand Coffee,43.251520,-79.878845,Café
3,Durand,43.250247,-79.875734,Bronzie's Place,43.250541,-79.871633,Italian Restaurant
4,Durand,43.250247,-79.875734,The Pheasant Plucker,43.251970,-79.870248,Pub


In [114]:
hamilton_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Beasley,10,10,10,10,10,10
Central Hamilton,67,67,67,67,67,67
Corktown,31,31,31,31,31,31
Durand,16,16,16,16,16,16
Gibson,4,4,4,4,4,4
Greenford,22,22,22,22,22,22
Hill Park,9,9,9,9,9,9
Kirkendall North,3,3,3,3,3,3
North End East,6,6,6,6,6,6


## 3. Analyze Each Neighborhood

In [17]:
# one hot encoding
hamilton_onehot = pd.get_dummies(hamilton_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hamilton_onehot['Neighborhood'] = hamilton_venues['Neighborhood'] 

first_col = hamilton_onehot.pop('Neighborhood')
hamilton_onehot.insert(0, 'Neighborhood', first_col)
hamilton_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,Bagel Shop,Bakery,Bank,Bar,Beer Store,Big Box Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Café,Chinese Restaurant,Coffee Shop,College Arts Building,College Baseball Diamond,College Basketball Court,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Ethiopian Restaurant,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Frozen Yogurt Shop,Garden,Gas Station,Gastropub,Gay Bar,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Lake,Lawyer,Library,Lighting Store,Liquor Store,Locksmith,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Multiplex,Museum,New American Restaurant,Nightclub,Noodle House,Park,Pharmacy,Pizza Place,Pub,Record Shop,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Mall,Skating Rink,Smoothie Shop,Snack Place,Soccer Stadium,Sports Bar,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,Durand,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Durand,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Durand,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Durand,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Durand,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Next, group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [18]:
hamilton_grouped = hamilton_onehot.groupby('Neighborhood').mean().reset_index()
hamilton_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,Bagel Shop,Bakery,Bank,Bar,Beer Store,Big Box Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Café,Chinese Restaurant,Coffee Shop,College Arts Building,College Baseball Diamond,College Basketball Court,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Ethiopian Restaurant,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Frozen Yogurt Shop,Garden,Gas Station,Gastropub,Gay Bar,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Lake,Lawyer,Library,Lighting Store,Liquor Store,Locksmith,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Multiplex,Museum,New American Restaurant,Nightclub,Noodle House,Park,Pharmacy,Pizza Place,Pub,Record Shop,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Mall,Skating Rink,Smoothie Shop,Snack Place,Soccer Stadium,Sports Bar,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,Beasley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000
1,Central Hamilton,0.029851,0.014925,0.000000,0.014925,0.000000,0.000000,0.000000,0.014925,0.029851,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.014925,0.029851,0.000000,0.029851,0.000000,0.119403,0.000000,0.000000,0.000000,0.014925,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.014925,0.059701,0.014925,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.014925,0.014925,0.000000,0.014925,0.000000,0.014925,0.0,0.029851,0.000000,0.029851,0.000000,0.014925,0.000000,0.0,0.00,0.000000,0.014925,0.0,0.000000,0.014925,0.029851,0.014925,0.014925,0.000000,0.014925,0.029851,0.000000,0.014925,0.014925,0.014925,0.059701,0.014925,0.044776,0.000000,0.000000,0.029851,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000
2,Corktown,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.032258,0.000000,0.000000,0.000000,0.032258,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.00000

#### Write a function to sort the venues in descending order.

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood.

In [120]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = hamilton_grouped['Neighborhood']

for ind in np.arange(hamilton_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hamilton_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beasley,Middle Eastern Restaurant,Pharmacy,Sushi Restaurant,Vietnamese Restaurant,Beer Store,Hostel,Theater,Fast Food Restaurant,Coffee Shop,Farm
1,Central Hamilton,Coffee Shop,Pub,Fast Food Restaurant,Restaurant,American Restaurant,Middle Eastern Restaurant,Indian Restaurant,Sandwich Place,Hotel,Nightclub
2,Corktown,Pub,Italian Restaurant,Park,Sandwich Place,Historic Site,Sports Bar,Gas Station,Fast Food Restaurant,Ethiopian Restaurant,Mexican Restaurant
3,Durand,Pub,Italian Restaurant,Café,Breakfast Spot,Fast Food Restaurant,Mexican Restaurant,Ethiopian Restaurant,Pharmacy,Pizza Place,Seafood Restaurant
4,Gibson,Restaurant,Coffee Shop,Gas Station,Library,Yoga Studio,Fish & Chips Shop,Dessert Shop,Diner,Discount Store,Dog Run


## 4. Cluster Neighborhoods

Add  the median rent to the dataframe

In [121]:
#avg_rent = rent_df[['Neighborhood','MedianRent']]
#avg_rent = avg_rent.sort_values(by = 'Neighborhood')
#df = pd.merge(neighborhoods_venues_sorted, avg_rent, on='Neighborhood', how='inner')
hamilton_merged = rent_df
hamilton_merged = hamilton_merged.rename(columns={'Neighbourhood':'Neighborhood'})
hamilton_merged

,Neighborhood,MedianRent,Latitude,Longitude
0,Durand,1300,43.250247,-79.875734
1,Central Hamilton,1675,43.256080,-79.872858
2,Beasley,1395,43.259204,-79.861012
3,Corktown,1375,43.250681,-79.868619
4,Gibson,1199,43.257866,-79.839098
5,Kirkendall North,1720,39.977308,-86.047118
6,Westdale South,720,43.261881,-79.905921
7,Riverdale West,1450,43.228332,-79.759820
8,Stinson,1310,43.246953,-79.852747
9,Strathcona,1272,43.265244,-79.883693


In [122]:
hamilton_merged= hamilton_merged.sort_values(by ='Neighborhood' )

In [123]:
hamilton_merged.reset_index(drop = True, inplace = True)

In [124]:
hamilton_merged

,Neighborhood,MedianRent,Latitude,Longitude
0,Beasley,1395,43.259204,-79.861012
1,Central Hamilton,1675,43.256080,-79.872858
2,Corktown,1375,43.250681,-79.868619
3,Durand,1300,43.250247,-79.875734
4,Gibson,1199,43.257866,-79.839098
5,Greenford,1650,43.225812,-79.769345
6,Hill Park,1115,-37.857526,175.679765
7,Kirkendall North,1720,39.977308,-86.047118
8,North End East,1200,43.269583,-79.857872
9,Raleigh,1348,43.204820,-79.842024


In [125]:
# set number of clusters
kclusters = 5

hamilton_grouped_clustering = hamilton_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hamilton_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 4, 0, 3, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [126]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
hamilton_merged = rent_df
hamilton_merged = hamilton_merged.rename(columns={'Neighbourhood':'Neighborhood'})

hamilton_merged = hamilton_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [130]:
hamilton_merged.dropna(subset=['Cluster Labels'],inplace=True)
hamilton_merged['Cluster Labels'] = hamilton_merged['Cluster Labels'].astype(int)
hamilton_merged

,Neighborhood,MedianRent,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Durand,1300,43.250247,-79.875734,0,Pub,Italian Restaurant,Café,Breakfast Spot,Fast Food Restaurant,Mexican Restaurant,Ethiopian Restaurant,Pharmacy,Pizza Place,Seafood Restaurant
1,Central Hamilton,1675,43.256080,-79.872858,0,Coffee Shop,Pub,Fast Food Restaurant,Restaurant,American Restaurant,Middle Eastern Restaurant,Indian Restaurant,Sandwich Place,Hotel,Nightclub
2,Beasley,1395,43.259204,-79.861012,0,Middle Eastern Restaurant,Pharmacy,Sushi Restaurant,Vietnamese Restaurant,Beer Store,Hostel,Theater,Fast Food Restaurant,Coffee Shop,Farm
3,Corktown,1375,43.250681,-79.868619,0,Pub,Italian Restaurant,Park,Sandwich Place,Historic Site,Sports Bar,Gas Station,Fast Food Restaurant,Ethiopian Restaurant,Mexican Restaurant
4,Gibson,1199,43.257866,-79.839098,2,Restaurant,Coffee Shop,Gas Station,Library,Yoga Studio,Fish & Chips Shop,Dessert Shop,Diner,Discount Store,Dog Run
5,Kirkendall North,1720,39.977308,-86.047118,1,Park,Golf Course,Dog Run,Yoga Studio,Fish & Chips Shop,Department Store,Dessert Shop,Diner,Discount Store,Ethiopian Restaurant
6,Westdale South,720,43.261881,-79.905921,0,Coffee Shop,Sandwich Place,Fried Chicken Joint,Burrito Place,Burger Joint,Flower Shop,Indie Movie Theater,Supermarket,Mediterranean Restaurant,Restaurant
7,Riverdale West,1450,43.228332,-79.759820,0,Fast Food Restaurant,Frozen Yogurt Shop,Pizza Place,Sandwich Place,Chinese Restaurant,Shopping Mall,Bus Station,Liquor Store,Lighting Store,Breakfast Spot
8,Stinson,1310,43.246953,-79.852747,3,Park,Trail,Coffee Shop,Lawyer,Locksmith,Yoga Studio,Fish & Chips Shop,Dessert Shop,Diner,Discount Store
9,Strathcona,1272,43.265244,-79.883693,0,Yoga Studio,Theater,Coffee Shop,Fast Food Restaurant,Park,Gas Station,Gastropub,Hotel,Pharmacy,History Museum


In [131]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, rent in zip(hamilton_merged['Latitude'], hamilton_merged['Longitude'], hamilton_merged['Neighborhood'], hamilton_merged['Cluster Labels'],hamilton_merged['MedianRent']):
    

    label = folium.Popup(str(poi) 
                         + ", Cluster:  " + str(cluster) 
                         + ", Average Rent: " + str(rent), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [135]:
hamilton_merged.groupby('Cluster Labels').mean()

,MedianRent,Latitude,Longitude
Cluster Labels,,,
0,1376.923077,36.697991,-60.556152
1,1720.000000,39.977308,-86.047118
2,1199.000000,43.257866,-79.839098
3,1310.000000,43.246953,-79.852747
4,1424.000000,43.226083,-79.807812


#### Cluster 1

In [134]:
hamilton_merged.loc[hamilton_merged['Cluster Labels'] == 0, hamilton_merged.columns[[0]+[1] + list(range(5, hamilton_merged.shape[1]))]]

,Neighborhood,MedianRent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Durand,1300,Pub,Italian Restaurant,Café,Breakfast Spot,Fast Food Restaurant,Mexican Restaurant,Ethiopian Restaurant,Pharmacy,Pizza Place,Seafood Restaurant
1,Central Hamilton,1675,Coffee Shop,Pub,Fast Food Restaurant,Restaurant,American Restaurant,Middle Eastern Restaurant,Indian Restaurant,Sandwich Place,Hotel,Nightclub
2,Beasley,1395,Middle Eastern Restaurant,Pharmacy,Sushi Restaurant,Vietnamese Restaurant,Beer Store,Hostel,Theater,Fast Food Restaurant,Coffee Shop,Farm
3,Corktown,1375,Pub,Italian Restaurant,Park,Sandwich Place,Historic Site,Sports Bar,Gas Station,Fast Food Restaurant,Ethiopian Restaurant,Mexican Restaurant
6,Westdale South,720,Coffee Shop,Sandwich Place,Fried Chicken Joint,Burrito Place,Burger Joint,Flower Shop,Indie Movie Theater,Supermarket,Mediterranean Restaurant,Restaurant
7,Riverdale West,1450,Fast Food Restaurant,Frozen Yogurt Shop,Pizza Place,Sandwich Place,Chinese Restaurant,Shopping Mall,Bus Station,Liquor Store,Lighting Store,Breakfast Spot
9,Strathcona,1272,Yoga Studio,Theater,Coffee Shop,Fast Food Restaurant,Park,Gas Station,Gastropub,Hotel,Pharmacy,History Museum
10,St. Clair,1250,Bar,Sushi Restaurant,Fast Food Restaurant,Pizza Place,Indian Restaurant,Dessert Shop,Sandwich Place,Yoga Studio,Deli / Bodega,Ethiopian Restaurant
12,Raleigh,1348,Wings Joint,Restaurant,Sandwich Place,Coffee Shop,Pharmacy,Yoga Studio,Fast Food Restaurant,Department Store,Dessert Shop,Diner
13,North End East,1200,Convenience Store,Restaurant,Coffee Shop,Skating Rink,Breakfast Spot,Brewery,Fish & Chips Shop,Dessert Shop,Diner,Discount Store


#### Cluster 2

In [136]:
hamilton_merged.loc[hamilton_merged['Cluster Labels'] == 1, hamilton_merged.columns[[0]+[1] + list(range(5, hamilton_merged.shape[1]))]]

,Neighborhood,MedianRent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Kirkendall North,1720,Park,Golf Course,Dog Run,Yoga Studio,Fish & Chips Shop,Department Store,Dessert Shop,Diner,Discount Store,Ethiopian Restaurant


#### Cluster 3

In [137]:
hamilton_merged.loc[hamilton_merged['Cluster Labels'] == 2, hamilton_merged.columns[[0]+[1] + list(range(5, hamilton_merged.shape[1]))]]

,Neighborhood,MedianRent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Gibson,1199,Restaurant,Coffee Shop,Gas Station,Library,Yoga Studio,Fish & Chips Shop,Dessert Shop,Diner,Discount Store,Dog Run


#### Cluster 4

In [138]:
hamilton_merged.loc[hamilton_merged['Cluster Labels'] == 3, hamilton_merged.columns[[0]+[1] + list(range(5, hamilton_merged.shape[1]))]]

,Neighborhood,MedianRent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Stinson,1310,Park,Trail,Coffee Shop,Lawyer,Locksmith,Yoga Studio,Fish & Chips Shop,Dessert Shop,Diner,Discount Store


#### Cluster 5

In [139]:
hamilton_merged.loc[hamilton_merged['Cluster Labels'] == 4, hamilton_merged.columns[[0]+[1] + list(range(5, hamilton_merged.shape[1]))]]

,Neighborhood,MedianRent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Rosedale,1424,Park,Convenience Store,Chinese Restaurant,Burrito Place,Yoga Studio,Fish & Chips Shop,Diner,Discount Store,Dog Run,Ethiopian Restaurant
